In [1]:
import sys
sys.path.append('../')

import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
import math
import functools

In [2]:
# Number of trees with n nodes where nodes can have at most k siblings
def num_trees_of_exactly_size_n(n, k):
    return math.comb(n*k, n) // ((k-1)*n + 1)

# Number of trees with n nodes, any k
def catalan_number(n):
    return num_trees_of_exactly_size_n(n, 2)

# Exclude trees such that n < k
def num_trees(n, k):
    return np.sum([num_trees_of_exactly_size_n(i, k) for i in range(1, n+1) if i >= k])

@functools.cache
def num_trees_bits(n, k):
    n_trees = num_trees(n, k)
    return math.floor(math.log2(n_trees) if n_trees > 0 else 0)

# Idiomatic industry-standard unit testing
assert num_trees_of_exactly_size_n(3, 2) == 5
assert num_trees_of_exactly_size_n(2, 2) == 2
assert num_trees_of_exactly_size_n(1, 2) == 1 # This tree will be excluded in the num_trees calculation because n < k
assert num_trees(3, 2) == 7
assert num_trees_bits(3, 2) == 2
assert num_trees(1, 2) == 0
assert num_trees_bits(1, 2) == 0
assert math.ceil(math.log2(num_trees_of_exactly_size_n(60, 14))) == 298 # From excel sheet

In [3]:
data = np.array([[num_trees_bits(n, 2)/n] for n in range(1, 1000)])

df = pd.DataFrame(data=data, columns=["catalan"])

fig = px.line(df)
fig.show()

In [4]:
N_MIN = 1
K_MIN = 2
N_RANGE = 200
K_RANGE = 200

n_values = np.array(range(N_MIN, N_RANGE+1), dtype=object)
k_values = np.array(range(K_MIN, K_RANGE+1), dtype=object)
z_values = np.array([[num_trees_bits(n, k) for n in n_values] for k in k_values])

fig = go.Figure(data=[go.Surface(
    x=n_values,
    y=k_values,
    z=z_values
)])

fig.update_layout(scene = dict(
                    xaxis_title="N",
                    yaxis_title="K",
                    zaxis_title="NUM_TREES_LOG"),
                    title="Number of trees", autosize=False,
                    width=800, height=800,
                    margin=dict(l=65, r=50, b=65, t=90))

fig.show()